In [5]:
import pinecone
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers 
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
load_dotenv()
import os

In [6]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
load_data = load_pdf("Data/")

In [8]:
# create text chuks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(load_data)
len(text_chunks)

11383

In [10]:
# download embedding model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_embeddings_model()

In [12]:
api_key = os.environ.get("PINECONE_API_KEY")
api_env = os.environ.get("PINECONE_API_ENV")

In [13]:
pinecone.Pinecone(api_key = api_key,
             environment = api_env)

index_name = "chatbot"

docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)


In [14]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
query = "What is capital of bihar."

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [15]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [17]:
llm=CTransformers(model="Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [18]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [19]:
user_input="What are Allergies"
result=qa({"query": user_input})
print("Response : ", result["result"])

In [20]:
user_input="What is fungal infection"
result=qa({"query": user_input})
print("Response : ", result["result"])